In [1]:

import joblib
import pandas as pd
import numpy as np
import os 
from itertools import compress
import random
from sklearn.model_selection import train_test_split

# Seed value
# Apparently you may use different seed values at each stage
seed_value = 0

# 1. Set the `PYTHONHASHSEED` environment variable at a fixed value
os.environ['PYTHONHASHSEED']=str(seed_value)

# 2. Set the `python` built-in pseudo-random generator at a fixed value
random.seed(seed_value)

# 3. Set the `numpy` pseudo-random generator at a fixed value
np.random.seed(seed_value)


merged_final=pd.read_csv('Dataset.csv')
Mort=pd.read_csv('Mortality_Data.csv')
# Prepara Data for Machine Learning
y=[]
for i in merged_final['c_s']=='S':
    if i:
        y.append(1)
    else:        
        y.append(0)


print(merged_final.columns)
print(Mort.columns)
merged_final=merged_final.merge(
    Mort[['icustay_id','hosp_mortality','mortality_90']],how='inner',on='icustay_id')

print(merged_final.columns)
X = merged_final[['filename','los','hosp_mortality','mortality_90','icustay_id','avg_Mu', 'avg_s2', 'avg_K', 'avg_srr_tot', 'avg_srr_vlf',
        'avg_srr_lf', 'avg_srr_hf', 'avg_srr_lfn', 'avg_srr_hfn',
        'avg_srr_lfhf', 'avg_sbp_tot', 'avg_sbp_vlf', 'avg_sbp_lf',
        'avg_sbp_hf', 'avg_sbp_lfn', 'avg_sbp_hfn', 'avg_scr_tot',
        'avg_scr_vlf', 'avg_scr_lf', 'avg_scr_hf', 'avg_scr_lfn', 'avg_scr_hfn',
        'avg_gain21_lf', 'avg_gain21_hf', 'avg_gain12_lf', 'avg_gain12_hf',
        'subject_id', 'NN20', 'pNN20', 'NN50', 'pNN50', 'logRMSSD',
        'SDSD', 'SD1', 'SD2', 'SD_ratio', 'SD_prod', 'TRI', 'TINN', 'AVSS',
        'SDSS', 'AVDD', 'SDDD', 'AVPP', 'SDPP', 'AVPTT', 'SDPTT',
        'RR_spect_slope', 'SS_spect_slope', 'DD_TOTPWR', 'DD_VLF', 'DD_LF', 'DD_HF',
        'DD_spect_slope', 'PP_TOTPWR', 'PP_VLF', 'PP_LF', 'PP_HF',
        'PP_spect_slope', 'PAT_TOTPWR', 'PAT_VLF', 'PAT_LF', 'PAT_HF',
        'PAT_spect_slope', 'Alpha1', 'H', 'SampEn', 'CorrDim',
        'LyapExp','age', 'vaso_flag', 'seda_flag', 'vent_flag','gender2','hypertension']] #,'hypertension'

diabetes=np.sum(merged_final[['diabetes_uncomplicated', 'diabetes_complicated']],axis=1)
#display(diabetes)
X['diabetes']=diabetes

X=X.rename(columns={'gender2':'gender'})

age=X.loc[:,'age']
X.loc[:,'age']= list(map(int, age))

# Filtering out age>89
print('Age')
X_filt=X.loc[X['age']<90]
y_filt=list(compress(y,X['age']<90 ))
display(X_filt['age'].loc[[x==1 for x in y_filt]].describe())
display(X_filt['age'].loc[[x==0 for x in y_filt]].describe())
#len(y_filt)

# LOS
print('LOS')
X_filt['los']=X_filt['los']/(60*60*24)
display(X_filt['los'].loc[[x==1 for x in y_filt]].describe())
display(X_filt['los'].loc[[x==0 for x in y_filt]].describe())

# Mortality
print('Mortality')
display(X_filt['hosp_mortality'].loc[[x==1 for x in y_filt]].sum()/sum(y_filt))
display(X_filt['hosp_mortality'].loc[[x==0 for x in y_filt]].sum()/sum([x==0 for x in y_filt]))

# 90D Mortality
print('Mortality 90')
display(X_filt['mortality_90'].loc[[x==1 for x in y_filt]].sum()/sum(y_filt))
display(X_filt['mortality_90'].loc[[x==0 for x in y_filt]].sum()/sum([x==0 for x in y_filt]))

# Gender
print('Gender')
display(X_filt['gender'].loc[[x==1 for x in y_filt]].sum()/sum(y_filt))
display(X_filt['gender'].loc[[x==0 for x in y_filt]].sum()/sum([x==0 for x in y_filt]))


# Stratified Train-Test Split (80/20%)
X_TRAIN, X_TEST, y_train, y_test = train_test_split(X_filt, y_filt,test_size=0.2,random_state=seed_value,stratify=y_filt)
display(X_TRAIN)

# Remove filename
X_train = X_TRAIN.drop(['filename','subject_id','icustay_id','los','hosp_mortality','mortality_90'],axis=1)
X_test = X_TEST.drop(['filename','subject_id','icustay_id','los','hosp_mortality','mortality_90'],axis=1)

display(X_train)
display(X_test)

print(X_train.columns)

# Check For Missings or Inf
X_train.isna().sum()
X_TRAIN.replace([np.inf, -np.inf], np.nan)
display(X_TRAIN[X_TRAIN.isna().any(axis=1)])


Index(['filename', 'los', 'hadm_id', 'icustay_id', 'avg_Mu', 'avg_s2', 'avg_K',
       'avg_srr_tot', 'avg_srr_vlf', 'avg_srr_lf', 'avg_srr_hf', 'avg_srr_lfn',
       'avg_srr_hfn', 'avg_srr_lfhf', 'avg_sbp_tot', 'avg_sbp_vlf',
       'avg_sbp_lf', 'avg_sbp_hf', 'avg_sbp_lfn', 'avg_sbp_hfn', 'avg_scr_tot',
       'avg_scr_vlf', 'avg_scr_lf', 'avg_scr_hf', 'avg_scr_lfn', 'avg_scr_hfn',
       'avg_gain21_lf', 'avg_gain21_hf', 'avg_gain12_lf', 'avg_gain12_hf',
       'subject_id', 'NN20', 'pNN20', 'NN50', 'pNN50', 'logRMSSD', 'SDSD',
       'SD1', 'SD2', 'SD_ratio', 'SD_prod', 'TRI', 'TINN', 'AVSS', 'SDSS',
       'AVDD', 'SDDD', 'AVPP', 'SDPP', 'AVPTT', 'SDPTT', 'RR_spect_slope',
       'SS_spect_slope', 'DD_TOTPWR', 'DD_VLF', 'DD_LF', 'DD_HF',
       'DD_spect_slope', 'PP_TOTPWR', 'PP_VLF', 'PP_LF', 'PP_HF',
       'PP_spect_slope', 'PAT_TOTPWR', 'PAT_VLF', 'PAT_LF', 'PAT_HF',
       'PAT_spect_slope', 'Alpha1', 'H', 'SampEn', 'CorrDim', 'LyapExp',
       'seda_flag', 'vaso_flag', 'ven

<ipython-input-1-1008c8d183e7>:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['diabetes']=diabetes


count    71.000000
mean     56.366197
std      17.180753
min      20.000000
25%      45.000000
50%      59.000000
75%      67.500000
max      86.000000
Name: age, dtype: float64

count    71.000000
mean     55.957746
std      15.638265
min      22.000000
25%      45.000000
50%      56.000000
75%      66.000000
max      88.000000
Name: age, dtype: float64

LOS


...

,filename,los,hosp_mortality,mortality_90,icustay_id,avg_Mu,avg_s2,avg_K,avg_srr_tot,avg_srr_vlf,...,SampEn,CorrDim,LyapExp,age,vaso_flag,seda_flag,vent_flag,gender,hypertension,diabetes
51,p047275-2131-03-07-17-29.mat,4.026192,0,0,265848,843.034091,0.168509,4467.564727,4.918474e+06,1.200396e+05,...,1.811811,-5.919922e-18,-0.117489,59,1,1,1,1,1,0
69,p059076-2189-06-24-16-43.mat,0.922662,0,0,274861,807.792445,0.230394,3291.406540,1.762419e+07,4.261824e+06,...,2.022977,1.504096e-01,-0.211945,59,0,0,0,1,0,0
34,p042397-2141-12-11-05-03.mat,1.381354,0,0,222348,797.660819,0.391401,1846.405861,6.507545e+08,8.157722e+07,...,2.246755,2.938818e-01,0.158442,20,0,1,1,0,0,0
57,p052238-2192-10-24-15-09.mat,0.923507,0,0,229626,1022.298159,0.370794,5325.316014,6.676602e+07,1.756411e+07,...,1.914477,1.495518e-01,-0.069385,50,0,1,0,1,0,0
39,p043086-2186-03-13-17-57.mat,3.751944,0,0,215887,491.454741,0.010839,11153.721072,7.936809e+04,9.117252e+03,...,0.659976,1.376537e-16,-0.032070,37,1,1,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,p068744-2161-06-24-19-25.mat,0.846736,0,1,213343,869.582494,0.147572,5009.535692,5.561697e+06,1.261820e+06,...,2.021417,1.100660e-01,-0.046052,85,0,1,0,1,1,1
55,p050561-2142-12-15-10-16.mat,2.054641,0,0,280356,946.489202,6.696974,131.275641,1.748625e+08,1.089375e+07,...,3.070741,3.722615e-01,0.455132,32,0,0,0,0,0,0
101,p074496-2142-06-17-17-11.mat,11.877245,0,0,285604,534.491638,0.007291,22115.066699,9.356432e+03,1.353011e+02,...,0.521452,3.028078e-17,0.097631,40,1,1,1,0,0,0
131,p092252-2141-04-11-14-25.mat,2.412650,0,0,278367,762.996319,0.015067,33089.756102,1.530984e+08,4.414272e+04,...,0.937615,9.592297e-02,-0.006335,74,1,1,1,1,1,0


,avg_Mu,avg_s2,avg_K,avg_srr_tot,avg_srr_vlf,avg_srr_lf,avg_srr_hf,avg_srr_lfn,avg_srr_hfn,avg_srr_lfhf,...,SampEn,CorrDim,LyapExp,age,vaso_flag,seda_flag,vent_flag,gender,hypertension,diabetes
51,843.034091,0.168509,4467.564727,4.918474e+06,1.200396e+05,2.591818e+06,9.810263e+05,0.435378,0.298059,9.072069,...,1.811811,-5.919922e-18,-0.117489,59,1,1,1,1,1,0
69,807.792445,0.230394,3291.406540,1.762419e+07,4.261824e+06,4.835514e+06,1.527134e+06,0.496107,0.270950,18.440177,...,2.022977,1.504096e-01,-0.211945,59,0,0,0,1,0,0
34,797.660819,0.391401,1846.405861,6.507545e+08,8.157722e+07,8.927771e+06,2.693038e+06,0.322125,0.400484,5.870738,...,2.246755,2.938818e-01,0.158442,20,0,1,1,0,0,0
57,1022.298159,0.370794,5325.316014,6.676602e+07,1.756411e+07,5.196859e+06,2.582407e+06,0.315788,0.481943,5.424955,...,1.914477,1.495518e-01,-0.069385,50,0,1,0,1,0,0
39,491.454741,0.010839,11153.721072,7.936809e+04,9.117252e+03,2.632291e+03,6.915962e+02,0.360342,0.116078,3.665137,...,0.659976,1.376537e-16,-0.032070,37,1,1,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,869.582494,0.147572,5009.535692,5.561697e+06,1.261820e+06,1.757322e+06,3.476523e+05,0.542341,0.268817,11.446193,...,2.021417,1.100660e-01,-0.046052,85,0,1,0,1,1,1
55,946.489202,6.696974,131.275641,1.748625e+08,1.089375e+07,2.256195e+07,9.909181e+07,0.233143,0.558295,2.466715,...,3.070741,3.722615e-01,0.455132,32,0,0,0,0,0,0
101,534.491638,0.007291,22115.066699,9.356432e+03,1.353011e+02,1.349609e+02,4.811901e+02,0.047611,0.172186,0.283681,...,0.521452,3.028078e-17,0.097631,40,1,1,1,0,0,0
131,762.996319,0.015067,33089.756102,1.530984e+08,4.414272e+04,3.645005e+03,8.963460e+03,0.094303,0.242464,0.736081,...,0.937615,9.592297e-02,-0.006335,74,1,1,1,1,1,0


,avg_Mu,avg_s2,avg_K,avg_srr_tot,avg_srr_vlf,avg_srr_lf,avg_srr_hf,avg_srr_lfn,avg_srr_hfn,avg_srr_lfhf,...,SampEn,CorrDim,LyapExp,age,vaso_flag,seda_flag,vent_flag,gender,hypertension,diabetes
135,1172.642054,1.166688,5378.942651,1.911778e+10,1.130087e+06,2.899212e+06,4.016035e+06,0.305670,0.532914,2.350002,...,1.935601,3.583111e-01,0.159563,69,0,0,0,1,0,0
138,956.039031,0.405911,3645.918585,9.164222e+07,3.363270e+06,2.015881e+06,2.108683e+06,0.333685,0.368357,5.539137,...,2.173606,1.757908e-01,-0.206578,38,0,0,0,1,0,0
89,1052.444713,1.288093,1172.530401,3.828707e+07,3.598871e+06,2.032028e+07,1.134057e+07,0.451940,0.450481,15.346165,...,2.765380,1.816230e-01,-0.085098,63,0,0,0,1,0,0
133,710.551955,0.055793,8101.848620,1.378275e+06,6.580457e+04,4.404193e+05,6.209874e+05,0.290446,0.425917,4.160110,...,1.272055,1.920237e-16,-0.210719,60,0,0,0,1,0,0
11,817.622099,0.019337,34175.562320,2.483270e+06,2.553801e+04,3.575246e+03,3.698045e+03,0.152639,0.203276,0.903774,...,1.178023,1.787042e-16,0.036912,85,0,0,0,1,1,0
107,546.694157,0.012035,14568.967095,1.073820e+08,1.699355e+04,6.647165e+03,2.959886e+04,0.169044,0.274671,2.239643,...,0.715259,4.339253e-17,0.004554,46,0,1,1,0,0,1
91,605.187630,0.020408,12027.181851,5.984848e+05,1.875610e+05,1.126678e+05,1.120215e+04,0.450930,0.230159,15.068881,...,0.853293,9.186633e-02,-0.003805,22,0,1,1,1,0,0
8,583.679292,0.009926,20370.411200,1.639605e+04,2.223677e+03,7.993246e+02,2.870578e+03,0.131962,0.322251,0.484184,...,0.692567,-4.224542e-18,-0.156671,51,0,1,1,1,0,0
36,604.192935,0.037730,6695.141369,5.108712e+05,3.369319e+04,1.331506e+05,1.586359e+04,0.443365,0.222511,8.580367,...,1.054970,6.335612e-17,-0.071754,57,1,0,0,0,0,0
108,595.062242,0.016000,14840.774544,4.191369e+05,1.301400e+05,6.342213e+04,3.389967e+03,0.453641,0.106515,13.464829,...,0.688778,-5.235690e-17,-0.028809,22,0,1,1,0,0,0


Index(['avg_Mu', 'avg_s2', 'avg_K', 'avg_srr_tot', 'avg_srr_vlf', 'avg_srr_lf',
       'avg_srr_hf', 'avg_srr_lfn', 'avg_srr_hfn', 'avg_srr_lfhf',
       'avg_sbp_tot', 'avg_sbp_vlf', 'avg_sbp_lf', 'avg_sbp_hf', 'avg_sbp_lfn',
       'avg_sbp_hfn', 'avg_scr_tot', 'avg_scr_vlf', 'avg_scr_lf', 'avg_scr_hf',
       'avg_scr_lfn', 'avg_scr_hfn', 'avg_gain21_lf', 'avg_gain21_hf',
       'avg_gain12_lf', 'avg_gain12_hf', 'NN20', 'pNN20', 'NN50', 'pNN50',
       'logRMSSD', 'SDSD', 'SD1', 'SD2', 'SD_ratio', 'SD_prod', 'TRI', 'TINN',
       'AVSS', 'SDSS', 'AVDD', 'SDDD', 'AVPP', 'SDPP', 'AVPTT', 'SDPTT',
       'RR_spect_slope', 'SS_spect_slope', 'DD_TOTPWR', 'DD_VLF', 'DD_LF',
       'DD_HF', 'DD_spect_slope', 'PP_TOTPWR', 'PP_VLF', 'PP_LF', 'PP_HF',
       'PP_spect_slope', 'PAT_TOTPWR', 'PAT_VLF', 'PAT_LF', 'PAT_HF',
       'PAT_spect_slope', 'Alpha1', 'H', 'SampEn', 'CorrDim', 'LyapExp', 'age',
       'vaso_flag', 'seda_flag', 'vent_flag', 'gender', 'hypertension',
       'diabetes'],
   

,filename,los,hosp_mortality,mortality_90,icustay_id,avg_Mu,avg_s2,avg_K,avg_srr_tot,avg_srr_vlf,...,SampEn,CorrDim,LyapExp,age,vaso_flag,seda_flag,vent_flag,gender,hypertension,diabetes


In [2]:
# Useful Functions

# Functions Definitions
# BOOTSTRAP
def bootstrapped_auc(y_true, y_pred, n_bootstraps = 1000, rng_seed = 42):

    import numpy as np
    from scipy.stats import sem
    from sklearn.metrics import precision_recall_curve,roc_curve, auc

    auroc_bootstrapped_scores=[]
    auprc_bootstrapped_scores=[]
    
    rng = np.random.RandomState(rng_seed)
    
    for i in range(n_bootstraps):
        # bootstrap by sampling with replacement on the prediction indices
        indices = rng.randint(0, len(y_pred), len(y_pred))
        #print(indices)
        if len(np.unique(y_true[indices])) < 2:
            # We need at least one positive and one negative sample for ROC AUC
            # to be defined: reject the sample
            continue

        roc_auc = roc_auc_score(y_true[indices], y_pred[indices])
        auroc_bootstrapped_scores.append(roc_auc)
        
        precision, recall, _ = precision_recall_curve(y_true[indices], y_pred[indices])
        prc_auc = auc(recall,precision)
        auprc_bootstrapped_scores.append(prc_auc)
        '''print("Bootstrap #{} ROC area: {:0.3f}".format(i + 1, score))'''

    sorted_auroc_scores = np.array(auroc_bootstrapped_scores)
    sorted_auroc_scores.sort()
    sorted_auprc_scores = np.array(auprc_bootstrapped_scores)
    sorted_auprc_scores.sort()
    
    # Computing the lower and upper bound of the 90% confidence interval
    # You can change the bounds percentiles to 0.025 and 0.975 to get
    # a 95% confidence interval instead.
    auroc_confidence_lower = sorted_auroc_scores[int(0.05 * len(sorted_auroc_scores))]
    auroc_confidence_upper = sorted_auroc_scores[int(0.95 * len(sorted_auroc_scores))]
    auprc_confidence_lower = sorted_auprc_scores[int(0.05 * len(sorted_auprc_scores))]
    auprc_confidence_upper = sorted_auprc_scores[int(0.95 * len(sorted_auprc_scores))]
    '''print("Confidence interval for the score: [{:0.3f} - {:0.3}]".format(
        confidence_lower, confidence_upper))'''
    
    return auroc_confidence_lower, auroc_confidence_upper, auprc_confidence_lower, auprc_confidence_upper


# Function that returns the threshold corresponding to the closest point to (1,0) on the ROC curve 
def find_best_th_roc(ytrain, temp_ytrain_hat):
    from sklearn.metrics import roc_curve
    if len(np.unique(temp_ytrain_hat))<3 or (max(np.diff(temp_ytrain_hat))>0.7):
        print('set to 0.5')
        best_th_roc=0.5
    else:
        fpr_train, tpr_train, th_roc_train = roc_curve(ytrain, temp_ytrain_hat)
        best_point_roc_x = np.array([0] * fpr_train.shape[0])
        best_point_roc_y = np.array([1] * tpr_train.shape[0])
        temp_x = (fpr_train - best_point_roc_x)
        temp_y = (tpr_train - best_point_roc_y)
        temp_sqrt = np.sqrt(np.square(temp_x) + np.square(temp_y))
        #plt.plot(temp_sqrt)
        #plt.show()
        index_min_temp_sqrt = np.argmin(temp_sqrt)
        best_th_roc = th_roc_train[index_min_temp_sqrt]
    return best_th_roc

# Function that returns the threshold corresponding to the minimum value of the abs(precision-recall) curve
def find_best_th_pr(ytrain, temp_ytrain_hat):
    from sklearn.metrics import precision_recall_curve
    if (len(np.unique(temp_ytrain_hat))<3) or (max(np.diff(temp_ytrain_hat))>0.7):
        print('set to 0.5')
        best_th_pr=0.5
    else:
        pre_train, rec_train, th_prc_train = precision_recall_curve(ytrain, temp_ytrain_hat)
        plt.plot((abs(pre_train - rec_train)))
        plt.show()
        print(temp_ytrain_hat)
        print(th_prc_train)
        print(th_prc_train)

        index_min_abs_prerec = np.argmin((abs(pre_train - rec_train)))
        best_th_pr = th_prc_train[index_min_abs_prerec]
    return best_th_pr

In [ ]:
# Feature Transformation, Feature Selection, Feature Scaler, Classifiers

from sklearn.pipeline import Pipeline
import os
from pathlib import Path
import joblib

from sklearn.metrics import precision_recall_curve, roc_auc_score, roc_curve, auc, confusion_matrix, classification_report
from matplotlib import pyplot as plt
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.model_selection import ShuffleSplit,StratifiedShuffleSplit
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
#Transformer
from sklearn.preprocessing import PowerTransformer, QuantileTransformer
#Feature Selector
from sklearn.feature_selection import SelectFromModel, VarianceThreshold, SelectKBest, SelectFdr, SelectFwe
from sklearn.feature_selection import mutual_info_classif, chi2, f_classif
from sklearn.linear_model import SGDClassifier
from sklearn.decomposition import PCA
from sklearn.feature_selection import RFECV


from sklearn.linear_model import LogisticRegression, ElasticNet
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier
import xgboost as xgb
from sklearn.neural_network import MLPClassifier

root_path=os.getcwd()
root_path=Path(root_path)

if os.path.isdir(root_path/('Plots')):
    print('Exist')
else:
    os.mkdir(root_path/('Plots'))

CV = 3
conf_ind=6 # Number of binary confoundings/features

feat_selector = ['passthrough',
                RFECV(LogisticRegression(), step=1, cv=10),
                RFECV(SVC(kernel="linear",random_state=seed_value), step=1, cv=10),
                VarianceThreshold(threshold=(.8 * (1 - .8))),
                Pipeline(steps=[('norm',MinMaxScaler())
                               ,('skb',SelectKBest(chi2, k=int(round(np.sqrt(X_train.shape[0])))))]),
                SelectKBest(mutual_info_classif, k=int(round(np.sqrt(X_train.shape[0])))),
                SelectFromModel(LogisticRegression(penalty='l1',solver='saga')) #LASSO penalty
                ]

Pipe_Yeo = Pipeline(steps=[('scl',StandardScaler()),('tr',PowerTransformer(method='yeo-johnson'))])
Pipe_Box = Pipeline(steps=[('scl',MinMaxScaler(feature_range=(1, 2))),('tr',PowerTransformer(method='box-cox'))])

# Try 2 Different Transformers (They Already Standardize Data)
transformer = ['passthrough',Pipe_Yeo,Pipe_Box 
               ,QuantileTransformer(output_distribution='normal',random_state=seed_value)
               ,QuantileTransformer(output_distribution='uniform',random_state=seed_value)]
scaler = ['passthrough', MinMaxScaler(), StandardScaler(), RobustScaler()]


classifiers = [KNeighborsClassifier(), # neighbors
    SVC(kernel='linear',probability=True, random_state=seed_value), #C
    SVC(kernel='rbf',probability=True, random_state=seed_value),
    MLPClassifier(activation='logistic',alpha=1, max_iter=1000),
    LogisticRegression(),
    DecisionTreeClassifier(random_state=seed_value),
    xgb.XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, max_delta_step=0,
       min_child_weight=1, missing=None, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=seed_value, subsample=1),
    ]

fsel_list=['None','RFECV_LR','RFECV_SVM','VAR_THR','KBEST_CHI2','KBEST_MI','LASSO']
transf_list=['None','Yeo-John','Box-Cox','Quant_Gaus','Quant_Unif']
scaler_list=['None','MinMax','StdScal','Robust']
clf_list=['KNN','SVC_lin','SVC_rbf','MLP','LR','TREE','XGB']

# Build Parameters Grid


pargrid=[{'classifier__n_neighbors':[2,3,5],
          'classifier__leaf_size':[3,10,30],
     'classifier__weights':['uniform', 'distance']},
    {'classifier__C':[0.1,0.5,1]},
    {'classifier__C':[0.1,0.5,1]},
    {'classifier__hidden_layer_sizes':[5,50,100],
     'classifier__learning_rate':['costant','adaptive']},
    {},
    {'classifier__criterion':['gini','entropy'],
        'classifier__max_depth':[2,3,5],
        'classifier__min_samples_split':[3,5],
        'classifier__min_samples_leaf':[1,2,3,5],
        'classifier__max_features':['sqrt','log2',None],
        'classifier__class_weight':[None,'balanced']}, # Decision Tree
    {'classifier__n_estimators':[100,250,500],
        'classifier__max_depth':[2,3,5],
        'classifier__learning_rate':[0.1,0.5,1]}
        ]

PIPES=[]
PIPES_DETAILS=[]

for transf,tf_lst in zip(transformer,transf_list):
    for feat_sel,fslist in zip(feat_selector,fsel_list):
                
        print(transf)
        print(fslist)
        column_transformer = ColumnTransformer(
            transformers=[('transformer', transf, X_train.columns[:-conf_ind])#
                          ,('no_transf', 'passthrough', X_train.columns[-conf_ind:])
                         ])
        pipe = Pipeline(steps=[('union', column_transformer)
                               ,('scal',MinMaxScaler()) # To give normalized data but without changing their  distribution
                               ,('feat_sel',feat_sel)])
        
        pipe.fit(X_train.copy(), y_train.copy())
        PIPES.append(pipe)
        PIPES_DETAILS.append([tf_lst,fslist])

print('feat_done')


# Initialize Vars
y_proba=[]
y_score=[]
best=[]
best_th_roc=[]

RESULTS_test=pd.DataFrame()

count=0
L=len(fsel_list)*len(clf_list)*len(transf_list)*len(scaler_list)

for pp,ppd in zip(PIPES,PIPES_DETAILS):
            for scal,sc_lst in zip(scaler,scaler_list):
                for classifier,pgrid,clf_lst in zip(classifiers,pargrid,clf_list):

                    count+=1
                    print('!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
                    print(str(count)+'/'+str(L))
                    print(ppd[0])
                    print(ppd[1])
                    print(sc_lst) 
                    print(clf_lst)
                    print('!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
                    print()
                    
                    # Build Pipeline 
                    pipe = Pipeline(steps=[('scaler',scal),
                                           ('classifier', classifier)])

                    # Parameters Grid For Grid Search
                    param_grid = pgrid

                    # Grid Search With Scoring F1 And Then Refit With All Passed Observation (10-Fold CV)
                    search = GridSearchCV(pipe, param_grid, cv=CV, n_jobs=-1, 
                                          scoring=['accuracy','f1','roc_auc','recall','precision'],refit='roc_auc')
                    
                    # Prepare the Data
                    X_tr_transf=pp.transform(X_train.copy())
                    X_te_transf=pp.transform(X_test.copy())
                    
                    # Fit Grid Search On Training Data with Hyperparametr Optimization In a 10-Fold Stratified CV
                    search.fit(X_tr_transf.copy(), np.array(y_train.copy()))

                    # Best Estimator Etraction
                    GSCV_results=pd.DataFrame(search.cv_results_) # Convert to Dataframe Grid Search CV Results

                    # Automatic Selection of Best Estimator (Inner Criteria: Maximum Average of Selected Refit Score )
                    best_pipe=search.best_estimator_
                    best_results=GSCV_results.loc[search.best_index_,:]
                    best_params = best_pipe.get_params()

                    # Extract Predicted Probabilities
                    y_proba_tr=best_pipe.predict_proba(X_tr_transf.copy())
                    y_proba.append(best_pipe.predict_proba(X_te_transf.copy())) # Compute Probabilities

                    # Save Best Pipeline
                    best.append(best_pipe)

                    # 10 Fold CV scores - Takes Too Long
                    '''
                    cv = StratifiedShuffleSplit(n_splits=10, test_size=0.2, random_state=seed_value)
                    cv_scores = cross_validate(search, X, y, cv=cv,
                                    scoring=('accuracy','f1','roc_auc'),
                                    return_train_score=True)
                    y_score.append(cv_scores)
                    print(cv_scores)
                    '''

                    # Plot Performance Measures at Each Cycle
                    # AUROC
                    fpr0, tpr0, _ = roc_curve(y_train, y_proba_tr[:, 1])
                    roc_auc0 = auc(fpr0, tpr0)
                    fpr, tpr, _ = roc_curve(y_test, y_proba[-1][:, 1])
                    roc_auc = auc(fpr, tpr)
                    # AUPRC
                    precision0, recall0, _ = precision_recall_curve(y_train, y_proba_tr[:, 1])
                    prc_auc0= auc(recall0,precision0)
                    precision, recall, _ = precision_recall_curve(y_test, y_proba[-1][:, 1])
                    prc_auc = auc(recall,precision)

                    # Optimal Threshold On Training Data
                    best_th_roc.append(find_best_th_roc(y_train, y_proba_tr[:,1]))
                    #best_th_roc.append(find_best_th_pr(y_train, best_pipe.predict_proba(X_train)[:,1]))

                    # Extract Optimal Threshold
                    cm=confusion_matrix(y_test, y_proba[-1][:,1]>best_th_roc[-1])
                    tn, fp, fn, tp=confusion_matrix(y_test, y_proba[-1][:,1]>best_th_roc[-1]).ravel()

                    # Compute Bootstrapped Confidence Intervals
                    auroc_cilw0, auroc_ciup0, auprc_cilw0, auprc_ciup0=bootstrapped_auc(np.array(y_train), y_proba_tr[:, 1],
                                                                                    n_bootstraps = 200, rng_seed = 0)
                    auroc_cilw, auroc_ciup, auprc_cilw, auprc_ciup=bootstrapped_auc(np.array(y_test), y_proba[-1][:, 1],
                                                                                    n_bootstraps = 200, rng_seed = 0)
                    # Plot AUROC and AUPRC
                    fig = plt.figure(figsize=(10,8))
                    ax1 = fig.add_subplot(1,2,1)
                    ax2 = fig.add_subplot(1,2,2)
                    lw = 2

                    # AUROC
                    ax1.plot(fpr0, tpr0, color='darkgreen',linestyle='-.',
                             lw=lw, label='Train ROC (AUC=%0.2f - CI=[%0.2f,%0.2f])' % (roc_auc0,auroc_cilw0,auroc_ciup0))
                    ax1.plot(fpr, tpr, color='darkgreen',
                             lw=lw, label='Test ROC (AUC=%0.2f - CI=[%0.2f,%0.2f])' % (roc_auc,auroc_cilw,auroc_ciup))
                    ax1.plot([0, 1], [0, 1], color='green', lw=lw, linestyle='--', label='ROC Ref = 0.5')
                    ax1.set_xlim([0.0, 1.0])
                    ax1.set_ylim([0.0, 1.05])
                    ax1.set_xlabel('False Positive Rate')
                    ax1.set_ylabel('Recall')
                    ax1.legend(loc="lower right")
                    ax1.set_title('Receiving Operating Characteristic Curve')
                    # AUPRC
                    ax2.plot(recall0, precision0, color='darkorange', linestyle='-.',
                             lw=lw, label='Train PRC (AUC=%0.2f - CI=[%0.2f,%0.2f])' % (prc_auc0,auprc_cilw0,auprc_ciup0))
                    ax2.plot(recall, precision, color='darkorange',
                             lw=lw, label='Test PRC (AUC=%0.2f - CI=[%0.2f,%0.2f])' % (prc_auc,auprc_cilw,auprc_ciup))
                    ax2.plot([0, 1], [sum(y)/len(y),sum(y)/len(y)],
                             color='orange', lw=lw, linestyle='--', label='PRC Ref = '+ str(round(sum(y)/len(y),2)))
                    ax2.set_xlim([0.0, 1.0])
                    ax2.set_ylim([0.0, 1.05])
                    ax2.set_xlabel('Recall')
                    ax2.set_ylabel('Precision')
                    ax2.legend(loc="lower right")
                    ax2.set_title('Precision-Recall Curve')

                    #Plot Layout
                    plt.tight_layout()


                    # Store Results in DataFrame 
                    dic={'Transformer':ppd[0],'Feat_Sel':ppd[1], 'Feat_Scal':sc_lst,'Classifier':clf_lst,'Pipeline':best_pipe,
                         'AUROC':round(roc_auc,4),'AUROC_CI':[round(auroc_cilw,4), round(auroc_ciup,4)],
                         'AUPRC':round(prc_auc,4),'AUPRC_CI':[round(auprc_cilw,4), round(auprc_ciup,4)],
                         'Threshold':best_th_roc[-1],'CM':cm,'TP':tp,'FP':fp,'TN':tn,'FN':fn}
                    df_temp=pd.DataFrame.from_dict(data=dic, orient='index').transpose()
                    df_temp
                    RESULTS_test=RESULTS_test.append(df_temp, ignore_index=True)
                    del df_temp

                    #display(RESULTS_test)

                    # Save Plot - Using the index of the 'RESULTS_Test' DataFrame as Identifier
                    plt.savefig(root_path/('Plots/Results Index - '+str(RESULTS_test.index[-1])+'.png'),dpi=300)

                    # Show Plot
                    #plt.show()
                    plt.close()
                        
                    joblib.dump(RESULTS_test,root_path / 'Results')

RESULTS_test_2=RESULTS_test
display(RESULTS_test_2)
del RESULTS_test

joblib.dump(RESULTS_test_2,root_path / 'Results')

passthrough
None
passthrough
RFECV_LR
passthrough
RFECV_SVM
passthrough
VAR_THR
passthrough
KBEST_CHI2
passthrough
KBEST_MI
passthrough
LASSO
Pipeline(steps=[('scl', StandardScaler()), ('tr', PowerTransformer())])
None


c:\users\maxim\sepsis_env\lib\site-packages\sklearn\linear_model\_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "


Pipeline(steps=[('scl', StandardScaler()), ('tr', PowerTransformer())])
RFECV_LR
Pipeline(steps=[('scl', StandardScaler()), ('tr', PowerTransformer())])
RFECV_SVM
Pipeline(steps=[('scl', StandardScaler()), ('tr', PowerTransformer())])
VAR_THR
Pipeline(steps=[('scl', StandardScaler()), ('tr', PowerTransformer())])
KBEST_CHI2
Pipeline(steps=[('scl', StandardScaler()), ('tr', PowerTransformer())])
KBEST_MI
Pipeline(steps=[('scl', StandardScaler()), ('tr', PowerTransformer())])
LASSO


c:\users\maxim\sepsis_env\lib\site-packages\sklearn\linear_model\_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "


Pipeline(steps=[('scl', MinMaxScaler(feature_range=(1, 2))),
                ('tr', PowerTransformer(method='box-cox'))])
None
Pipeline(steps=[('scl', MinMaxScaler(feature_range=(1, 2))),
                ('tr', PowerTransformer(method='box-cox'))])
RFECV_LR
Pipeline(steps=[('scl', MinMaxScaler(feature_range=(1, 2))),
                ('tr', PowerTransformer(method='box-cox'))])
RFECV_SVM
Pipeline(steps=[('scl', MinMaxScaler(feature_range=(1, 2))),
                ('tr', PowerTransformer(method='box-cox'))])
VAR_THR
Pipeline(steps=[('scl', MinMaxScaler(feature_range=(1, 2))),
                ('tr', PowerTransformer(method='box-cox'))])
KBEST_CHI2
Pipeline(steps=[('scl', MinMaxScaler(feature_range=(1, 2))),
                ('tr', PowerTransformer(method='box-cox'))])
KBEST_MI
Pipeline(steps=[('scl', MinMaxScaler(feature_range=(1, 2))),
                ('tr', PowerTransformer(method='box-cox'))])
LASSO


...

QuantileTransformer(random_state=0)
LASSO
feat_done
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
1/980
None
None
None
KNN
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!



c:\users\maxim\sepsis_env\lib\site-packages\sklearn\preprocessing\_data.py:2367: UserWarning: n_quantiles (1000) is greater than the total number of samples (113). n_quantiles is set to n_samples.
  warnings.warn("n_quantiles (%s) is greater than the total number "
c:\users\maxim\sepsis_env\lib\site-packages\sklearn\linear_model\_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "


set to 0.5
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
2/980
None
None
None
SVC_lin
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
3/980
None
None
None
SVC_rbf
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
4/980
None
None
None
MLP
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
5/980
None
None
None
LR
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
6/980
None
None
None
TREE
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

set to 0.5
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
7/980
None
None
None
XGB
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

set to 0.5
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
8/980
None
None
MinMax
KNN
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

set to 0.5
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

c:\users\maxim\sepsis_env\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


set to 0.5
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
27/980
None
None
Robust
TREE
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!



In [ ]:
import matplotlib.pyplot as plt


DF=joblib.load('Results')
df=DF.copy()

TT=df.sort_values(by='AUROC', ascending=False).reset_index()
display(TT.iloc[0:10,:])


TT_best=TT.iloc[0:10,:]
TT_best.loc[:,'AUROC']=[round(x,2) for x in TT_best.loc[:,'AUROC']]
TT_best.loc[:,'AUPRC']=[round(x,2) for x in TT_best.loc[:,'AUPRC']]
TT_best['AUROC_CI']=[[round(x[0],2),round(x[1],2)] for x in TT_best['AUROC_CI']]
TT_best['AUPRC_CI']=[[round(x[0],2),round(x[1],2)] for x in TT_best['AUPRC_CI']]
TT_best['Recall']=[round(x,2) for x in TT_best['TP']/(TT_best['TP']+TT_best['FN'])]
TT_best['Precision']=[round(x,2) for x in TT_best['TP']/(TT_best['TP']+TT_best['FP'])]
TT_best['Specificity']=[round(x,2) for x in TT_best['TN']/(TT_best['TN']+TT_best['FP'])]
TT_best['F1']=[round(x,2) for x in 2*TT_best['TP']/(2*TT_best['TP']+TT_best['FP']+TT_best['FN'])]
display(TT_best)

auroc_cil=[x[0] for x in TT_best['AUROC_CI']]
TT_best['AUROC_CI_L']=auroc_cil
auroc_ciu=[x[1] for x in TT_best['AUROC_CI']]
TT_best['AUROC_CI_U']=auroc_ciu
auprc_cil=[x[0] for x in TT_best['AUPRC_CI']]
TT_best['AUPRC_CI_L']=auprc_cil
auprc_ciu=[x[1] for x in TT_best['AUPRC_CI']]
TT_best['AUPRC_CI_U']=auprc_ciu
auroc=[]
for x0,x1,x2 in zip(TT_best['AUROC'],TT_best['AUROC_CI_L'],TT_best['AUROC_CI_U']):
    auroc.append(str(x0)+' +/- ('+str(x1)+'-'+str(x2)+')')
auprc=[]
for x0,x1,x2 in zip(TT_best['AUPRC'],TT_best['AUPRC_CI_L'],TT_best['AUPRC_CI_U']):
    auprc.append(str(x0)+' +/- ('+str(x1)+'-'+str(x2)+')')
    
TT_best['auroc']=auroc
TT_best['auprc']=auprc

TT_best['Threshold']=[round(x,2) for x in TT_best['Threshold']]

TT_best[['Transformer','Feat_Sel','Feat_Scal',
         'Classifier','auroc','auprc','Threshold','Recall',
         'Precision','Specificity','F1']].to_csv('Best_Results.csv',header=True,index=False)

#display(TT)

#plt.bar()
plt.figure(figsize=(40,60))
imp,names = zip(*sorted(zip(abs(TT['Pipeline'][1]['classifier'].coef_[0]),X_train.columns)))
plt.barh(range(len(names)), imp, align='center')
plt.yticks(range(len(names)), names,fontsize=40)
plt.xticks(fontsize=40)
plt.title('Feature Importance',fontsize=40)
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import joblib
import pandas as pd
DF=joblib.load('Results_FEATSEL_V3_2_try_2')
df=DF.copy()

# Classifier
clf=df['Classifier'].unique()
print(clf)
xx = []
for x in clf:
    xx.append(df['AUROC'].loc[df['Classifier']==x])
    
plt.boxplot(xx, labels=clf)
plt.xticks(rotation='vertical')
plt.title('AUROC With Different Classifiers')
plt.savefig('AUROC_Class0',dpi=300)
plt.show()

# Remove SVC_gamma
df=df.loc[df['Classifier']!='SVC_rbf']


# Transformer
tr=df['Transformer'].unique()
print(tr)
xx = []
for x in tr:
    xx.append(df['AUROC'].loc[df['Transformer']==x])
    
plt.boxplot(xx, labels=tr)
plt.xticks(rotation='vertical')
plt.title('AUROC With Different Transformers')
plt.savefig('AUROC_Transf',dpi=300)
plt.show()

# Feature Selection
sel=df['Feat_Sel'].unique()
print(sel)
xx = []
for x in sel:
    xx.append(df['AUROC'].loc[df['Feat_Sel']==x])
    
plt.boxplot(xx, labels=sel)
plt.xticks(rotation='vertical')
plt.title('AUROC With Different Feature Selection Methods')
plt.savefig('AUROC_Fsel',dpi=300)
plt.show()

# Feature Scaling
scal=df['Feat_Scal'].unique()
print(scal)
xx = []
for x in scal:
    xx.append(df['AUROC'].loc[df['Feat_Scal']==x])
    
plt.boxplot(xx, labels=scal)
plt.xticks(rotation='vertical')
plt.title('AUROC With Different Feature Selection')
plt.savefig('AUROC_Fscal',dpi=300)
plt.show()

# Classifier
clf=df['Classifier'].unique()
print(scal)
xx = []
for x in clf:
    xx.append(df['AUROC'].loc[df['Classifier']==x])
    
plt.boxplot(xx, labels=clf)
plt.xticks(rotation='vertical')
plt.title('AUROC With Different Classifiers')
plt.savefig('AUROC_Class',dpi=300)
plt.show()